# Example Notebook

In [ ]:
# example.ipynb

from chemoecology_tools.core import GCMSExperiment
from chemoecology_tools.analysis import calculate_enrichment_table, perform_nmds
from chemoecology_tools.visualization import plot_nmds

In [ ]:
USER_PROVIDED_CHEMICAL_METADATA_PATH = "../example_data/user_provided_chemical_metadata.yaml"
ABUNDANCE_DATA_PATH = "../example_data/LBAL23_relative_abundance.csv"
USER_PROVIDED_SAMPLE_METADATA_PATH = "../example_data/LBAL23_metadata.csv"
ID_COLUMN = "Bee #"

In [ ]:
# Load experimental data with metadata
experiment = GCMSExperiment.from_files(
    abundance_path=ABUNDANCE_DATA_PATH,
    metadata_path=USER_PROVIDED_SAMPLE_METADATA_PATH,
    id_col=ID_COLUMN,
    fetch_pubchem=True,
    filter_dict={
        "CHC File Located": ["NO", "Bleed"],
        "Quality": ["poor", "contaminated"],
        "Nest Code": ["E-JS-030", "E-JS-033", "E-JS-037", "E-KO-140", "E-KO-143"]
    },
    user_chemical_metadata=USER_PROVIDED_CHEMICAL_METADATA_PATH
)

In [ ]:
print(experiment)

In [ ]:
# Filter and process data
filtered = experiment.filter_trace_compounds(threshold=0.005)
relative = filtered.calculate_relative_abundance()

In [ ]:
filtered.metadata_df.head()

In [ ]:
import pandas as pd

meta_mask = filtered.metadata_df["Caste"].isin(["Sol_r", "Sol_nr"])
chem_mask = pd.Series([
    filtered.get_chemical_property(c, "class") == "Alkane"
    for c in filtered.chemical_cols
], index=filtered.chemical_cols)

In [ ]:
filtered_exp = experiment.filter(
    metadata_mask=meta_mask,
    chemical_mask=chem_mask
)
print(filtered_exp)

In [ ]:
nmds_df = perform_nmds(relative)
nmds_df.head()

In [ ]:
# Create plots with different groupings
# By caste
fig1 = plot_nmds(
    experiment=relative,
    nmds_coords=nmds_df,
    group_col="Caste",
    title="Chemical Profiles by Caste"
)

In [ ]:
# Generate enrichment table
enrichment_table = calculate_enrichment_table(
    experiment=filtered,
    group_column='Caste',
    class_column='class',
    alpha=0.05
)

In [ ]:
enrichment_table.head()